# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-Coder-480B-A35B-Instruct',
 '/moonshotai/Kimi-K2-Instruct',
 '/Qwen/Qwen3-235B-A22B-Instruct-2507',
 '/bosonai/higgs-audio-v2-generation-3B-base',
 '/mistralai/Voxtral-Mini-3B-2507',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/akhaliq/anycoder',
 '/spaces/multimodalart/wan2-1-fast',
 '/spaces/smola/higgs_audio_v2',
 '/spaces',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/common-pile/caselaw_access_project',
 '/datasets/microsoft/rStar-Coder',
 '/datasets/interstellarninja/hermes_reasoning_tool_use',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook'

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
Welcome Cohere on the Hub 🔥
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
Qwen/Qwen3-Coder-480B-A35B-Instruct
Updated
about 7 hours ago
•
2.71k
•
554
moonshotai/Kimi-K2-Instruct
Updated
2 days ago
•
230k
•

In [23]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nWelcome Cohere on the Hub 🔥\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-Coder-480B-A35B-Instruct\nUpdated\nabout 7 hours ago\n•\n2.71k\n•\n554\nmoonshotai/Kimi-K2-Instruct\nUpdated\n2 days ago\n•\n230k\n•\n1.79k\nQwen/Qwen3-235B-A22B-Instruct-2507\nUpdated\n3 days ago\n•\n8.23k\n•\n453\nbosonai/higgs-audio-v2-generation-3B-base\nUp

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## **About Us**

**Hugging Face** is at the forefront of the AI revolution, building a collaborative community that encompasses developers, researchers, and AI enthusiasts. Our mission is to democratize artificial intelligence by making cutting-edge machine learning models, datasets, and applications easily accessible.

**Website:** [huggingface.co](https://huggingface.co)

---

## **Our Offerings**

### **Models, Datasets & Spaces**
- **Models**: Explore over **1M+ machine learning models** ranging from language generation to image processing.
- **Datasets**: Access **250k+ datasets** that are continuously updated and maintained by our community.
- **Spaces**: Run and collaborate on **10.8k+ interactive applications** that harness AI capabilities for users.

With an array of tools like **Transformers** for deep learning, **Diffusers** for generative modeling, and more, we provide an open-source stack that accelerates machine learning deployment and exploration.

---

## **Enterprise Solutions**

Over **50,000 organizations**, including industry giants like Google, Microsoft, and Amazon, utilize our enterprise-grade platform that prioritizes security and support. Our offerings include:

- Optimized Inference Endpoints
- Team collaboration tools
- Single Sign-On and access control features

---

## **Company Culture**

At Hugging Face, we embody a **community-first mindset** where collaboration is key. Our culture fosters:

- **Innovation**: Encouraging experimentation and the open sharing of findings.
- **Inclusivity**: Welcoming AI enthusiasts from diverse backgrounds to contribute and grow.
- **Support**: Providing resources, documentation, and technical assistance to our community members.

We prioritize transparency and sustainability, striving to build trust through our open-source contributions.

---

## **Careers at Hugging Face**

Join our dynamic team where you can make a real impact!

### **Why Work With Us?**
- Work alongside passionate individuals dedicated to advancing the field of AI.
- Engage in a collaborative and flexible working environment.
- Opportunities for growth in a rapidly evolving industry.

### **Open Positions**
Explore various roles across engineering, data science, product management, and more. We are eager to find innovative thinkers who are ready to shape the future of AI with us.

---

## **Join Us**

Whether you’re a prospective customer looking to harness the power of AI, an investor interested in pioneering technology, or a talent eager to join a passionate team, Hugging Face welcomes you.

**Get started today**: [Sign Up](https://huggingface.co/join)

Together, let’s build the future of AI! 🚀

--- 

**Hugging Face - The AI community building the future.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face Brochure

## Welcome to Hugging Face 🌟
We are the AI community building the future. Hugging Face is a collaborative platform for machine learning, where developers, researchers, and enthusiasts come together to share models, datasets, and applications that push the boundaries of what's possible with AI.

---

## What We Offer 🛠️
- **Models**: Explore over 1M+ AI models, ranging from state-of-the-art natural language processing to cutting-edge image recognition and beyond.
- **Datasets**: Access and share 250k+ datasets that support various ML tasks, helping you find the right data to train your models.
- **Spaces**: Run and deploy ML applications seamlessly with our hosted Spaces feature.
- **Enterprise Solutions**: Our advanced platform supports over 50,000 organizations, offering custom enterprise-grade security, resources, and scalable compute options.

## Who We Serve 🤝
Our community includes a diverse array of users from various sectors:
- **Enterprises**: Companies like Amazon, Google, Microsoft, and Meta use Hugging Face to enhance their AI capabilities.
- **Startups & Non-profits**: Organizations of all sizes leverage our tools and resources to innovate and solve real-world challenges.

---

## Company Culture 🌈
At Hugging Face, we foster an inclusive environment where collaboration and open-source contributions are highly valued. Our community-driven approach ensures that everyone, from novices to experts, can share, learn, and grow together. We believe in giving back to the AI community, empowering people through knowledge and access to resources.

## Careers @ Hugging Face 🌟
Join a passionate team dedicated to advancing machine learning. We offer:
- Opportunities to work on cutting-edge technologies
- A supportive culture that prioritizes well-being and work-life balance
- Continuous learning and development opportunities

Explore our current job openings and be part of a mission-driven company that's shaping the future of AI.

---

## Connect with Us 🌐
Join our community and start your journey with Hugging Face!
- **Website**: [huggingface.co](https://huggingface.co)
- **GitHub**: [github.com/huggingface](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [linkedin.com/company/huggingface](https://linkedin.com/company/huggingface)

---
Join us in building the future of AI!



In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face: The AI Community Building the Future! 🤖❤️

Are you ready to embark on an epic journey of AI adventures? Well, buckle up because here at Hugging Face, we’re not just cuddling machines; we’re launching them into the future! 🚀

---

## Who Are We? 

Imagine a place where minds meet, ideas flourish, and AI models strut around like they own the playground. That's us! We provide a collaborative home for the machine learning community to share models, datasets, and applications. So whether you're a data wizard, a coding sorcerer, or just someone who loves cute robots, there’s a space for you here!

---

## What We Offer 

### 💡 Models Galore!
With over **1 million models**, you could say we’re kind of a big deal! Some trending stars this week include:
- **Qwen/Qwen3-Coder-480B-A35B-Instruct** – Because who doesn’t want a coder with 480 billion parameters?
- **moonshotai/Kimi-K2-Instruct** – Surprisingly not a new flavor of ice cream.
- **bosonai/higgs-audio-v2-generation-3B-base** – Perfect for when you want to generate tunes without the talent!

### 📂 Datasets, Datasets, Datasets!
Over **250,000 datasets** to choose from! That’s more choices than your last trip to the snack aisle! 

### 🏗️ Spaces for Creativity!
Join **10,800 applications** running in our Hugging Spaces! From generating modern web pages to animating your pet’s dreams – if you can dream it, you can create it!

---

## Culture: The Fun in Functional 🎉

At Hugging Face, we believe in creating a joyful work environment where innovation thrives. Our "brainstorming" sessions often include snacks, playful debates about AI ethics, and the occasional robot dance-off. Yes, you read that right. We believe in the power of (not-so) serious fun!

How do we keep our community engaged? Well, we don't just do meetings; we host **virtual coffee breaks** where ideas flow like a good brew! ☕ And don't get us started on our **team-building retreats**—trust us, attempting to teach robots salsa dancing is a sight to behold!

---

## Join Us! Your Next Adventure Awaits 🚪

### Explore Careers!
Looking to work with the brightest minds in AI? We’re always on the lookout for talents who are:
- Passionate about AI 🧠
- Skilled in coding (or just have a knack for debugging)
- Eager to create a community of inclusive AI enthusiasts

From data scientists battling data dragons to engineers sculpting models like Michelangelo, every role here is essential!

---

## Customers We Love ❤️

Our playground isn’t just for individual aficionados; organizations like Amazon, Google, and Microsoft vibe with us! More than **50,000 organizations** use our platform because who can resist supporting a community that makes AI feel like a warm hug?

---

## Get Started Today!

Ready to dive into the world of models, datasets, and a community that feels like home? 
- **SIGN UP** and start collaborating, coding, and creating today! 
- **Check out our pricing** and discover why we’re the best-value AI platform around!

---

So, come join the teddy bear brigade of AI innovation at Hugging Face! Together, we’ll build the future, one hug at a time! 🤗

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>